In [ ]:
"""
References to functions in the code:
integer_modulo(x,n) # x mod n
polynomial_modulo(p,n,v) # p mod n
polynomial_division(p,d,v) # p / d, return [quotient, remainder]
def GF(p,q,v)
degree_of_q_polynomials(p,q,v)
find_irreducible_polynomial(p,q,v) # irreducible polynomials of degree q


---- find compositional irreducible polynomials mod 2 (composed with monomials)(exclude constants) ----
monomial_mod2(q,v) # excludes the constant 1, up to degree q-1
combination_monomials_mod2(q,v)
composition_polynomials_mod2_by_monomials(irre,q,v) #output all compositional reducible polynomials of degree q over irreducible polynomial irre of degree q+1
find_compositional_irreducible_mod2_by_monomials(irre,q,v)


---- find compositional irreducible polynomials mod 2 (composed with all polynomials in the field)(exclude constants) ----
component_polynomials_mod2(x,q,v) # return x(h(v)) for one polynomial x and all h in GF(2,q)
combination_polynomials_mod2(x,q,v)
composition_polynomials_mod2_by_allPoly(irre,q,v) #output all compositional reducible polynomials of degree q over irreducible polynomial irre of degree q+1
find_compositional_irreducible_mod2_by_allPoly(irre,q,v)

"""

In [20]:
def integer_modulo(x,n):
    """
    Outputs result of modulo operation of two integers, where the input integer x
    modulo the input positive integer n. 

    EXAMPLES:

    ::
        
        sage: x = 35; n = 2; 
        sage: integer_modulo(x,n)
        1
        sage: x = 7; n = 7;
        sage: integer_modulo(x,n)
        0
        sage: x = 3; n = 0;
        sage: integer_modulo(x,n)
        "n must be a positive integer"

        


    AUTHORS:
    - Alan Li and Wenxuan Lu
    """    
    if n<=0:
        return "n must be a positive integer"
    r=x
    while r>=n:
        r=r-n
    while r<0:
        r=r+n
    return r

In [21]:
def polynomial_modulo(p,n,v):
    """
    Outputs result of polynomial modulo integer n, where the input univariate polynomial p
    in the variable input v modulo the input positive integer n. 

    EXAMPLES:

    ::
        
        sage: p = x^2+2*x+1; n = 2; v = var('x') 
        sage: polynomial_modulo(p,n,v)
        [x^2 + 1]
        sage: p = x^2+5*x+1; n = 3; v = var('x')
        sage: polynomial_modulo(p,n,v)
        [x^2 + 2*x + 1]
        sage: p = 4*x^3+3*x^2+2*x+1; n =2 ; v =var('x')
        sage: polynomial_modulo(p,n,v)
        [x^2 + 1]

        


    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    if n<0:
        return "n must be a positive integer"
    newp=0
    for i in range (len(p.coefficients(v))):
        newp=newp+integer_modulo(p.coefficients(v)[i][0],n)*v^p.coefficients(v)[i][1]
    return newp

In [22]:
# p is the dividend; d is the divisor; v is the variable; output[q,r], quotient and remainder
def polynomial_division(p,d,v):
    """
    Outputs result of the Euclidean division. Where the input
    univariate polynomial p in the variable input v is the dividend
    and the input univariate polynomial d in the input variable v is 
    the divisor.
    
    Since it's a function for polynomial division, constant c must
    be written as c*x^0.


    EXAMPLES:

    ::
        
        sage: p = 4*x^3+3*x^2+2*x+1; d = 5*x^2+3*x+7; v = var('x') 
        sage: polynomial_division(p,d,v)
        [4/5*x + 3/25, -99/25*x + 4/25] 
        sage: p = x^2+2*x+1; d = 2*x^0; v = var('x')
        sage: polynomial_division(p,d,v)
        [1/2*x^2+x+1/2, 0]
        sage: p = x^2+x+1; d =x^5 ; v =var('x')
        sage: polynomial_division(p,d,v)
        [0, x^2+x+1]
        sage: p = x^2+x+1; d =0*x^5 ; v =var('x')
        sage: polynomial_division(p,d,v)
        "invalid input, please input a polynomial"
        


    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    if p.degree(v)>=1 and d.degree(v)>=1:
        p=p.expand()
        d=d.expand()
        q=0
        r=p
        while p.degree(v)>=d.degree(v):
            L=(p.coefficient(v^p.degree(v))/d.coefficient(v^d.degree(v)))*v^(p.degree(v)-d.degree(v))
            p=p-(L*d).expand()
            q=q+L
        r=p
        return [q,r]
    else:
        return [p,d]

In [23]:
def GF(p,q,v):
    """
    Outputs elements of finite field or Galois field GF(p^q). Where output all the polynomials
    in the varible v that have coefficients less than p and degree of at most q-1.
    

    EXAMPLES:

    ::
        
        sage: p = 2; q = 3; v = var('x') 
        sage: GF(p,q,v)
        [0, 1, x, x + 1, x^2, x^2 + 1, x^2 + x, x^2 + x + 1]
        sage: p = 5; q = 2; v = var('x')
        sage: GF(p,q,v)
        [0, 1, 2, 3, 4, x, x + 1, x + 2, x + 3, x + 4, 2*x, 2*x + 1, 2*x + 2, 2*x + 3, 2*x + 4,
        3*x, 3*x + 1, 3*x + 2, 3*x + 3, 3*x + 4, 4*x, 4*x + 1, 4*x + 2, 4*x + 3, 4*x + 4]
        sage: p = 1; q =1 ; v =var('x')
        sage: GF(p,q,v)
        [0]

        


    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    poly=[];
    if q!=1:
        for i in range(p):
            new=[i*v^(q-1) + k for k in GF(p,q-1,v)]
            poly=poly+new
    else:
        for i in range(p):
            poly.append(i)
    return poly

In [24]:
def degree_of_q_polynomials(p,q,v):
    """
    Outputs all the polynomials in the varible v that with coefficients less than p, and degree of q
    

    EXAMPLES:

    ::
        
        sage: p = 2; q = 3; v = var('x') 
        sage: degree_of_q_polynomials(p,q,v)
        [x^3, x^3 + 1, x^3 + x, x^3 + x + 1, x^3 + x^2, x^3 + x^2 + 1, x^3 + x^2 + x, 
         x^3 + x^2 + x + 1]
        sage: p = 2; q = 4; v = var('x')
        sage: degree_of_q_polynomials(p,q,v)
        [x^4, x^4 + 1, x^4 + x, x^4 + x + 1, x^4 + x^2, x^4 + x^2 + 1, x^4 + x^2 + x, x^4 + x^2 + x + 1, x^4 + x^3,
         x^4 + x^3 + 1, x^4 + x^3 + x, x^4 + x^3 + x + 1, x^4 + x^3 + x^2, x^4 + x^3 + x^2 + 1,
         x^4 + x^3 + x^2 + x, x^4 + x^3 + x^2 + x + 1]
        sage: p = 1; q =1 ; v =var('x')
        sage: degree_of_q_polynomials(p,q,v)
        []

        


    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    if q<1:
        return "q should >= 1"
    field=GF(p,q,v)
    poly=[]
    for i in range(1,p):
        for j in field:
            poly.append(i*v^q+j)
    return poly

In [25]:
def find_irreducible_polynomial(p,q,v):
    """
    Outputs elements of all irrducible polynomials of finite field or Galois field GF(c^d). 
    irrducible polynomials are defined as f(v) in input variable v,
    such that if there do not exist two nonconstant polynomials g(v) and h(v) in input
    variable v with integer coefficients such that f(v)=g(v)h(v).
    Where all polynomial has positive coefficients less thaninput integer c, 
    and degree less than or equal to input d
    

    EXAMPLES:

    ::
        
        sage: p = 2; q = 2; v = var('x') 
        sage: find_irreducible_polynomial(2,2,x)
        [x^2 + x + 1]
        sage: p = 2; q = 4; v = var('x')
        sage: find_irreducible_polynomial(2,4,x)
        [x^4 + x + 1, x^4 + x^3 + 1, x^4 + x^3 + x^2 + x + 1]
        sage: p = 2; d =5 ; v =var('x')
        sage: find_irreducible_polynomial(2,5,x)
        [x^5 + x^2 + 1, x^5 + x^3 + 1, x^5 + x^3 + x^2 + x + 1, x^5 + x^4 + x^2 + x + 1,
         x^5 + x^4 + x^3 + x + 1, x^5 + x^4 + x^3 + x^2 + 1]
        


    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    field = GF(p,q,v)
    for i in range(p): # remove all constants, which cannot produce a polynomial of degree q
        field.remove(i)
    
    poly=[] # the same as poly = degree_of_q_polynomials(p,q,v), but more efficient
    for i in range(1,p):
        for j in field:
            poly.append(i*v^q+j)
    
    for i in field:
        for j in range((q - i.degree(v)) * p, len(field)): # polynomial with smaller degree cannot produce a polynomial of degree q
            result = polynomial_modulo((i * field[j]).expand(),p,v)
            if result.degree(v) == q and result in poly: # remove reducible polynomial and avoid multiple removing of the same polynomial
                poly.remove(result) 
    return poly

In [26]:
def monomial_mod2(q,v):
    """
    Outputs a list of monomials in variable v with coefficient 1 and degree at most q-1(exclude the constants)
    
    
    EXAMPLES:

    ::
        
        sage: q = 4; v = var('x')
        sage: monomial_mod2(q,v)
        [x, x^2, x^3]
        sage: q = 1; v =var('x')
        sage: monomial_mod2(q,v)
        []

        

    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    poly=[]
    #for i in range(q): #if don't exclude the constants
    for i in range(1,q):
        new=v^i
        poly.append(new)
    return poly

In [27]:
from itertools import combinations 
def combination_monomials_mod2(q,v):
    """
    Outputs a list of all combinations of monomials in variable v with coefficient 1 and degrees from 1 to q-1(exclude the constants)
    
    
    EXAMPLES:

    ::
        
        sage: q = 4; v = var('x')
        sage: combination_monomials_mod2(q,v)
        [(x,), (x^2,), (x^3,), (x, x^2), (x, x^3), (x^2, x^3), (x, x^2, x^3)]
        sage: q = 1; v =var('x')
        sage: combination_monomials_mod2(q,v)
        []

        

    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    component=monomial_mod2(q,v)
    poly=[]
    for i in range(1,len(component)+1):
        new=list(combinations(component, i))
        poly=poly+new
    return poly

In [28]:
#  Outputs all compositional reducible polynomials of degree n over GF(2^q) with respect to the irreducible polynomial irre of degree q+1 in variable v. 

# need new examples

def composition_polynomials_mod2_by_monomials(irre,q,v):
    """
    Outputs all compositional reducible polynomials of degree n over GF(2^q) with respect to the irreducible polynomial irre of degree q+1 in variable v. 
    f(x) is said to be compositional reducible polynomials if there exist g(x) and a list of monomial [h1(x), h2(x), ...] for some irreducible polynomial t(x)
    such that f(x)=(g(h1(x))+g(h2(x))+g(h3(x))+...) mod t(x)

    EXAMPLES:

    ::
        
        sage:v=var('v')
        sage:irre = v^4 + v + 1
        sage:composition_polynomials_mod2_by_monomials(irre,3,v)
        []
        sage:v=var('v')
        sage:irre = v^4 + v^3 + 1
        sage:composition_polynomials_mod2_by_monomials(irre,3,v)
        [v^3 + 1, v^3 + v^2 + 1, v^3, v^3 + v + 1, v^3 + v^2]
        sage:v=var('v')
        sage:irre = v^4 + v^3 + v^2 + v + 1
        sage:composition_polynomials_mod2_by_monomials(irre,3,v)
        [v^3 + v^2 + v + 1, v^3 + v + 1, v^3 + v^2 + v, v^3 + v^2 + 1, v^3 + v]
        
        
        
    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    field=GF(2,q,v)
    field.remove(0); # remove the constants
    field.remove(1);
    poly=[] # to store all reducible polynomials
    compo=combination_monomials_mod2(q,v)

    for i in field:
        for j in range(len(compo)):
            new = 0
            for k in compo[j]:
                new = new+i.subs(v==k)
            new = polynomial_modulo(new,2,v)
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
            new = polynomial_division(new,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
            new = polynomial_modulo(new,2,v)
            if new.degree(v) == q and new not in poly:
                poly.append(new)
    return poly

In [29]:
# Outputs all compositional irreducible polynomials of degree n over GF(2^q) with respect to the irreducible polynomial irre of degree q+1 in variable v. 

# need new examples

def find_compositional_irreducible_mod2_by_monomials(irre,q,v):
    """
    Outputs all compositional irreducible polynomials of degree n over GF(2^q) with respect to the irreducible polynomial irre of degree q+1 in variable v. 
    f(x) is said to be compositional irreducible polynomials if there doesn't exist g(x) and a list of monomial [h1(x), h2(x), ...] for some irreducible polynomial t(x)
    such that f(x)=(g(h1(x))+g(h2(x))+g(h3(x))+...) mod t(x)
    

    EXAMPLES:

    ::
        
        sage:v=var('v')
        sage:irre = v^4 + v + 1
        sage:find_compositional_irreducible_mod2_by_monomials(irre,3,v)
        [v^3, v^3 + 1, v^3 + v, v^3 + v + 1, v^3 + v^2, v^3 + v^2 + 1, v^3 + v^2 + v, v^3 + v^2 + v + 1]
        sage:v=var('v')
        sage:irre = v^4 + v^3 + 1
        sage:find_compositional_irreducible_mod2_by_monomials(irre,3,v)
        [v^3 + v, v^3 + v^2 + v, v^3 + v^2 + v + 1]
        sage:v=var('v')
        sage:irre = v^4 + v^3 + v^2 + v + 1
        sage:find_compositional_irreducible_mod2_by_monomials(irre,3,v)
        [v^3, v^3 + 1, v^3 + v^2]
        
        

    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    field=degree_of_q_polynomials(2,q,v)
    poly=[] # to store all irreducible polyonmials
    poly2=composition_polynomials_mod2_by_monomials(irre,q,v)
    for i in field:
        if i not in poly2:
            poly.append(i)
    return poly

In [35]:
def component_polynomials_mod2(x,q,v):
    """
    Outputs all composition polynomials resulting from composing polynomial x with all polynomials in GF(2,q) in variable v; Constants are excluded.
    
    
    EXAMPLES:

    ::
        
        sage: x = v^2+1; q = 3; v = var('v')
        sage: component_polynomials_mod2(x,q,v)
        [v^2 + 1, v^2, v^4 + 1, v^4, v^4 + v^2 + 1, v^4 + v^2]
        sage: x = v^3+v+1; q = 4; v =var('v')
        sage: component_polynomials_mod2(x,q,v)
        [v^3 + v + 1, v^3 + v^2 + 1, v^6 + v^2 + 1, v^6 + v^4 + 1, v^6 + v^5 + v^4 + v^3 + v^2 + v + 1,
         v^6 + v^5 + v^3 + v^2 + 1, v^9 + v^3 + 1, v^9 + v^6 + 1, v^9 + v^7 + v^5 + v + 1, v^9 + v^7 + v^6 + v^5 + v^3 + v^2 + 1,
         v^9 + v^8 + v^7 + v^6 + v^3 + v^2 + 1, v^9 + v^8 + v^7 + v^4 + 1, v^9 + v^8 + v^6 + v^4 + v^2 + v + 1,
         v^9 + v^8 + v^3 + v^2 + 1]

        

    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    
    
    field=GF(2,q,v)
    # remove all constant elements, delete the next two lines if want to include constants
    field.remove(0)
    field.remove(1)
    
    poly=[] # to store all component polynomials
    for i in field:
        new=x.subs(v==i)
        new=polynomial_modulo(new,2,v)
        if new not in poly:
            poly.append(new)
    return poly

In [31]:
from itertools import combinations 
def combination_polynomials_mod2(x,q,v):
    
    """
    Outputs a list of tuples of all composition polynomials resulting from composing polynomial x with all polynomials in GF(2,q) in variable v; Constants are excluded.
    
    
    EXAMPLES:

    ::
        
        sage: x = v^2+1; q = 3; v = var('v')
        sage: combination_polynomials_mod2(x,q,v)
        [(v^2 + 1,), (v^2,), (v^4 + 1,), (v^4,), (v^4 + v^2 + 1,), (v^4 + v^2,), (v^2 + 1, v^2), (v^2 + 1, v^4 + 1),
         (v^2 + 1, v^4), (v^2 + 1, v^4 + v^2 + 1), (v^2 + 1, v^4 + v^2), (v^2, v^4 + 1), (v^2, v^4), (v^2, v^4 + v^2 + 1),
         (v^2, v^4 + v^2), (v^4 + 1, v^4), (v^4 + 1, v^4 + v^2 + 1), (v^4 + 1, v^4 + v^2), (v^4, v^4 + v^2 + 1),
         (v^4, v^4 + v^2), (v^4 + v^2 + 1, v^4 + v^2), (v^2 + 1, v^2, v^4 + 1), (v^2 + 1, v^2, v^4), (v^2 + 1, v^2, v^4 + v^2 + 1),
         (v^2 + 1, v^2, v^4 + v^2), (v^2 + 1, v^4 + 1, v^4), (v^2 + 1, v^4 + 1, v^4 + v^2 + 1), (v^2 + 1, v^4 + 1, v^4 + v^2),
         (v^2 + 1, v^4, v^4 + v^2 + 1), (v^2 + 1, v^4, v^4 + v^2), (v^2 + 1, v^4 + v^2 + 1, v^4 + v^2), (v^2, v^4 + 1, v^4),
         (v^2, v^4 + 1, v^4 + v^2 + 1), (v^2, v^4 + 1, v^4 + v^2), (v^2, v^4, v^4 + v^2 + 1), (v^2, v^4, v^4 + v^2),
         (v^2, v^4 + v^2 + 1, v^4 + v^2), (v^4 + 1, v^4, v^4 + v^2 + 1), (v^4 + 1, v^4, v^4 + v^2), (v^4 + 1, v^4 + v^2 + 1, v^4 + v^2),
         (v^4, v^4 + v^2 + 1, v^4 + v^2), (v^2 + 1, v^2, v^4 + 1, v^4), (v^2 + 1, v^2, v^4 + 1, v^4 + v^2 + 1),
         (v^2 + 1, v^2, v^4 + 1, v^4 + v^2), (v^2 + 1, v^2, v^4, v^4 + v^2 + 1), (v^2 + 1, v^2, v^4, v^4 + v^2),
         (v^2 + 1, v^2, v^4 + v^2 + 1, v^4 + v^2), (v^2 + 1, v^4 + 1, v^4, v^4 + v^2 + 1), (v^2 + 1, v^4 + 1, v^4, v^4 + v^2),
         (v^2 + 1, v^4 + 1, v^4 + v^2 + 1, v^4 + v^2), (v^2 + 1, v^4, v^4 + v^2 + 1, v^4 + v^2), (v^2, v^4 + 1, v^4, v^4 + v^2 + 1),
         (v^2, v^4 + 1, v^4, v^4 + v^2), (v^2, v^4 + 1, v^4 + v^2 + 1, v^4 + v^2), (v^2, v^4, v^4 + v^2 + 1, v^4 + v^2),
         (v^4 + 1, v^4, v^4 + v^2 + 1, v^4 + v^2), (v^2 + 1, v^2, v^4 + 1, v^4, v^4 + v^2 + 1),
         (v^2 + 1, v^2, v^4 + 1, v^4, v^4 + v^2), (v^2 + 1, v^2, v^4 + 1, v^4 + v^2 + 1, v^4 + v^2),
         (v^2 + 1, v^2, v^4, v^4 + v^2 + 1, v^4 + v^2), (v^2 + 1, v^4 + 1, v^4, v^4 + v^2 + 1, v^4 + v^2),
         (v^2, v^4 + 1, v^4, v^4 + v^2 + 1, v^4 + v^2), (v^2 + 1, v^2, v^4 + 1, v^4, v^4 + v^2 + 1, v^4 + v^2)]

        

    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    
    compo=component_polynomials_mod2(x,q,v)
    poly=[]
    for i in range(1,len(compo)+1):
        new=list(combinations(compo, i))
        poly=poly+new
    return poly

In [32]:
def composition_polynomials_mod2_by_allPoly(irre,q,v):
    """
    Outputs all compositional reducible polynomials of degree n over GF(2^q) with respect to the irreducible polynomial irre of degree q+1 in variable v. 
    f(x) is said to be compositional reducible polynomials if there exist g(x) and a list of polynomials in GF(2^q) [h1(x), h2(x), ...] for some irreducible polynomial t(x)
    such that f(x)=(g(h1(x))+g(h2(x))+g(h3(x))+...) mod t(x)

    EXAMPLES:

    ::
        
        sage:v=var('v')
        sage:irre=v^4 + v^3 + v^2 + v + 1
        sage:composition_polynomials_mod2_by_allPoly(irre,3,v)
        [v^3 + v^2 + v + 1, v^3 + v^2 + v, v^3 + v + 1, v^3 + v, v^3 + v^2 + 1, v^3 + v^2]
        sage:v=var('v')
        sage:irre = v^5 + v^2 + 1
        sage:composition_polynomials_mod2_by_allPoly(irre,4,v)
        [v^4, v^4 + 1, v^4 + v^2, v^4 + v^2 + 1, v^4 + v^3 + v, v^4 + v^3 + v + 1, v^4 + v^3 + v^2 + v,
         v^4 + v^3 + v^2 + v + 1, v^4 + v, v^4 + v^2 + v, v^4 + v + 1, v^4 + v^2 + v + 1, v^4 + v^3 + 1,
         v^4 + v^3 + v^2, v^4 + v^3, v^4 + v^3 + v^2 + 1]
        
        
    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    field=GF(2,q,v)
    # remove all constants
    field.remove(0)
    field.remove(1)
    
    poly=[] # to store all reducible polynomials
    for i in field:
        compo=combination_polynomials_mod2(i,q,v)
        for j in range(len(compo)):
            new=0
            for k in range(len(compo[j])):
                new=new+compo[j][k]
            new=polynomial_modulo(new,2,v)
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
            new = polynomial_division(new,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
            new = polynomial_modulo(new,2,v)
            if new.degree(v) == q and new not in poly:
                poly.append(new)
    return poly

In [33]:
def find_compositional_irreducible_mod2_by_allPoly(irre,q,v):
        """
    Outputs all compositional irreducible polynomials of degree n over GF(2^q) with respect to the irreducible polynomial irre of degree q+1 in variable v. 
    f(x) is said to be compositional irreducible polynomials if there doesn't exist g(x) and a list of monomial [h1(x), h2(x), ...] for some irreducible polynomial t(x)
    such that f(x)=(g(h1(x))+g(h2(x))+g(h3(x))+...) mod t(x)
    

    EXAMPLES:

    ::
        
        sage:v=var('v')
        sage:irre = v^4 + v + 1
        sage:find_compositional_irreducible_mod2_by_allPoly(irre,3,v)
        [v^3, v^3 + 1]
        sage:v=var('v')
        sage:irre = v^5 + v^2 + 1
        sage:find_compositional_irreducible_mod2_by_allPoly(irre,4,v)
        []
        

    AUTHORS:
    - Alan Li and Wenxuan Lu
    """
    field=degree_of_q_polynomials(2,q,v)
    poly=[] # to store all irreducible polyonmials
    poly2=composition_polynomials_mod2_by_allPoly(irre,q,v)
    for i in field:
        if i not in poly2:
            poly.append(i)
    return poly